### Imports

In [7]:
import pyomo.environ as pyo
import pandas as pd
import numpy as np
from datetime import date, datetime
import ast
import os
import matplotlib.pyplot as plt

### Define DA model with Flexibility Options and Storage Support

In [ ]:
class DAFOModel:
    def __init__(self, num_periods=24, num_scenarios=5, num_generators=0, num_tiers=4, num_storage=0):
        """
        Initialize the DAFO model with flexible set dimensions.
        
        Parameters:
        -----------
        num_periods : int, optional
            Number of time periods (default: 24)
        num_scenarios : int, optional
            Number of scenarios (default: 5)
        num_generators : int, optional
            Number of generators (if None, determined from data)
        num_tiers : int, optional
            Number of tiers (default: 4)
        num_storage : int, optional
            Number of storage units (if None, determined from data)
        """
        self.num_periods = num_periods
        self.num_scenarios = num_scenarios
        self.num_generators = num_generators
        self.num_tiers = num_tiers
        self.num_storage = num_storage
        
        self.model = pyo.AbstractModel()
        self._define_sets()
        self._define_parameters()
        self._define_variables()
        self._define_objective()
        self._define_constraints()
    
    def _define_sets(self):
        self.model.T = pyo.RangeSet(1, self.num_periods)    # Set of time periods
        self.model.S = pyo.RangeSet(1, self.num_scenarios)  # Set of scenarios
        self.model.R = pyo.RangeSet(1, self.num_tiers)     # Set of tiers
        
        # Generator set
        if self.num_generators > 0:
            self.model.G = pyo.RangeSet(1, self.num_generators)
        else:
            self.model.G = pyo.Set()
        
        # Storage set
        if self.num_storage > 0:
            self.model.B = pyo.RangeSet(1, self.num_storage)
        else:
            self.model.B = pyo.Set()
    
    def _define_parameters(self):
        # General Parameters
        self.model.VC = pyo.Param(self.model.G)           # Variable cost
        self.model.VCUP = pyo.Param(self.model.G)         # Variable cost up
        self.model.VCDN = pyo.Param(self.model.G)         # Variable cost down
        self.model.CAP = pyo.Param(self.model.G)          # Capacity
        self.model.REDA = pyo.Param(self.model.T)         # Maximum DA RE for each hour
        self.model.DEMAND = pyo.Param(self.model.T)       # Electricity demand per hour
        self.model.D1 = pyo.Param(within=pyo.NonNegativeIntegers)    # Linear cost coefficient for demand slack
        self.model.D2 = pyo.Param(within=pyo.NonNegativeIntegers)    # Quadratic cost coefficient for demand slack

        # Parameters specific to the FO
        self.model.RR = pyo.Param(self.model.G)                      # Ramp rate
        self.model.RE = pyo.Param(self.model.S, self.model.T)        # Renewable generation at each scenario and time
        self.model.PEN = pyo.Param(within=pyo.NonNegativeIntegers)   # Penalty for inadequate flexibility up
        self.model.PENDN = pyo.Param(within=pyo.NonNegativeIntegers) # Penalty for inadequate flexibility down
        self.model.smallM = pyo.Param(within=pyo.NonNegativeReals)   # Parameter for alternative optima
        self.model.probTU = pyo.Param(self.model.R)                  # Probability of exercise FO up
        self.model.probTD = pyo.Param(self.model.R)                  # Probability of exercise FO down

        # Storage Parameters
        self.model.E_MAX = pyo.Param(self.model.B)        # Maximum energy capacity
        self.model.P_MAX = pyo.Param(self.model.B)        # Maximum power capacity
        self.model.ETA_CH = pyo.Param(self.model.B)       # Charging efficiency
        self.model.ETA_DCH = pyo.Param(self.model.B)      # Discharging efficiency
        self.model.E0 = pyo.Param(self.model.B)           # Initial state of charge
        self.model.E_FINAL = pyo.Param(self.model.B)      # Required final state of charge
        self.model.STORAGE_COST = pyo.Param(self.model.B) # Storage operating cost per MWh
    
    def _define_variables(self):
        # Energy and reserve variables
        self.model.d = pyo.Var(self.model.T, domain=pyo.NonNegativeReals)      # Demand slack
        self.model.rgDA = pyo.Var(self.model.T, domain=pyo.NonNegativeReals)   # DA renewables schedule
        self.model.du = pyo.Var(self.model.S, self.model.T)                    # Demand uncertainty
        
        # Variables dependent on generator set
        if len(self.model.G) > 0 or isinstance(self.model.G, pyo.RangeSet):
            self.model.xDA = pyo.Var(self.model.G, self.model.T, domain=pyo.NonNegativeReals)  # DA energy schedule
            # generator FO Variables
            self.model.hsu = pyo.Var(self.model.R, self.model.G, self.model.T, domain=pyo.NonNegativeReals)  # Supply FO up
            self.model.hsd = pyo.Var(self.model.R, self.model.G, self.model.T, domain=pyo.NonNegativeReals)  # Supply FO down
            
        # FO Variables independent of generators and storage
        self.model.hdu = pyo.Var(self.model.R, self.model.T, domain=pyo.NonNegativeReals)     # Demand FO up
        self.model.hdd = pyo.Var(self.model.R, self.model.T, domain=pyo.NonNegativeReals)     # Demand FO down
        self.model.sdu = pyo.Var(self.model.R, self.model.T, domain=pyo.NonNegativeReals)     # Self-supply FO up
        self.model.sdd = pyo.Var(self.model.R, self.model.T, domain=pyo.NonNegativeReals)     # Self-supply FO down
        self.model.y = pyo.Var(self.model.S, self.model.T, domain=pyo.NonNegativeReals)       # Auxiliary variable

        # Variables dependent on storage set
        if len(self.model.B) > 0 or isinstance(self.model.B, pyo.RangeSet):
            # Storage Variables
            self.model.e = pyo.Var(self.model.B, self.model.T, domain=pyo.NonNegativeReals)     # Energy level
            self.model.p_ch = pyo.Var(self.model.B, self.model.T, domain=pyo.NonNegativeReals)  # Charging power
            self.model.p_dch = pyo.Var(self.model.B, self.model.T, domain=pyo.NonNegativeReals) # Discharging power
            # Storage FO Variables
            self.model.bsu = pyo.Var(self.model.R, self.model.B, self.model.T, domain=pyo.NonNegativeReals)  # Storage FO up
            self.model.bsd = pyo.Var(self.model.R, self.model.B, self.model.T, domain=pyo.NonNegativeReals)  # Storage FO down
    
    def _define_objective(self):
        # Objective function
        def obj_expression(m):
            objective_terms = []
            
            # Energy costs - generators
            if hasattr(m, 'xDA') and len(m.G) > 0:
                objective_terms.append(sum(m.VC[g] * m.xDA[g,t] for g in m.G for t in m.T))
            
            # Flexibility costs - generators
            if hasattr(m, 'hsu') and len(m.G) > 0:
                objective_terms.append(sum(m.probTU[r] * m.VCUP[g] * m.hsu[r,g,t] for g in m.G for r in m.R for t in m.T))
                objective_terms.append(-sum(m.probTD[r] * m.VCDN[g] * m.hsd[r,g,t] for g in m.G for r in m.R for t in m.T))
            
            # Self-supply flexibility costs
            objective_terms.append(sum(m.probTU[r] * m.PEN * m.sdu[r,t] for r in m.R for t in m.T))
            objective_terms.append(-sum(m.probTD[r] * m.PENDN * m.sdd[r,t] for r in m.R for t in m.T))
            
            # Storage related costs
            if hasattr(m, 'bsu') and len(m.B) > 0:
                # Storage flexibility costs
                objective_terms.append(sum(m.probTU[r] * m.STORAGE_COST[b] * m.bsu[r,b,t] for b in m.B for r in m.R for t in m.T))
                objective_terms.append(-sum(m.probTD[r] * m.STORAGE_COST[b] * m.bsd[r,b,t] for b in m.B for r in m.R for t in m.T))
                # Storage operation costs
                objective_terms.append(sum(m.STORAGE_COST[b] * (m.p_ch[b,t] + m.p_dch[b,t]) for b in m.B for t in m.T))
            
            # Auxiliary costs
            objective_terms.append(sum(m.y[s,t] for s in m.S for t in m.T) * m.smallM)
            
            # Demand response costs
            objective_terms.append(sum(0.2 * m.D1 * (m.d[t] + m.du[s,t]) for s in m.S for t in m.T))
            objective_terms.append(0.2 * m.D2 * sum((m.d[t] + m.du[s,t]) * (m.d[t] + m.du[s,t]) for s in m.S for t in m.T))
            
            return sum(objective_terms)

        self.model.OBJ = pyo.Objective(rule=obj_expression)
    
    def _define_constraints(self):
        # Define constraints - Numbering of constraints follows paper
        # Energy balance for each hour
        def DA_energy_balance(model, t):
            gen_sum = 0
            storage_sum = 0
            
            if hasattr(model, 'xDA'):
                gen_sum = sum(model.xDA[g,t] for g in model.G) if len(model.G) > 0 else 0
                
            if hasattr(model, 'p_dch') and hasattr(model, 'p_ch'):
                storage_sum = sum(model.p_dch[b,t] - model.p_ch[b,t] for b in model.B) if len(model.B) > 0 else 0
            
            return (gen_sum + 
                    model.rgDA[t] + 
                    storage_sum + 
                    model.d[t] == model.DEMAND[t])
        
        self.model.Con3 = pyo.Constraint(self.model.T, rule=DA_energy_balance)

        # Flexibility balance for each hour
        def DA_flexup_balance(model, r, t):
            gen_flex_up = 0
            storage_flex_up = 0
            
            if hasattr(model, 'hsu'):
                gen_flex_up = sum(model.hsu[r,g,t] for g in model.G) if len(model.G) > 0 else 0
                
            if hasattr(model, 'bsu'):
                storage_flex_up = sum(model.bsu[r,b,t] for b in model.B) if len(model.B) > 0 else 0
            
            return (gen_flex_up + storage_flex_up == model.hdu[r,t])
        
        self.model.Con4UP = pyo.Constraint(self.model.R, self.model.T, rule=DA_flexup_balance)

        def DA_flexdn_balance(model, r, t):
            gen_flex_down = 0
            storage_flex_down = 0
            
            if hasattr(model, 'hsd'):
                gen_flex_down = sum(model.hsd[r,g,t] for g in model.G) if len(model.G) > 0 else 0
                
            if hasattr(model, 'bsd'):
                storage_flex_down = sum(model.bsd[r,b,t] for b in model.B) if len(model.B) > 0 else 0
            
            return (gen_flex_down + storage_flex_down == model.hdd[r,t])
        
        self.model.Con4DN = pyo.Constraint(self.model.R, self.model.T, rule=DA_flexdn_balance)

        # Flexibility demand for each scenario and hour
        def DA_flex_demand(model, s, t):
            return (-model.du[s,t] + 
                    sum(model.hdd[r,t] + model.sdd[r,t] for r in model.R if r <= s-1) -
                    sum(model.hdu[r,t] + model.sdu[r,t] for r in model.R if r >= s) == 
                    model.RE[s,t] - model.rgDA[t])
        
        self.model.Con6 = pyo.Constraint(self.model.S, self.model.T, rule=DA_flex_demand)

        # Add generator constraints only if generators exist
        if hasattr(self.model, 'xDA') and (len(self.model.G) > 0 or isinstance(self.model.G, pyo.RangeSet) or self.num_generators is None):
            # Inter-temporal constraints
            def ramp_rate_up(model, g, t):
                if t == 1:
                    return pyo.Constraint.Skip
                return model.xDA[g,t] - model.xDA[g,t-1] <= model.RR[g]
            
            self.model.ramp_up = pyo.Constraint(self.model.G, self.model.T, rule=ramp_rate_up)

            def ramp_rate_down(model, g, t):
                if t == 1:
                    return pyo.Constraint.Skip
                return model.xDA[g,t-1] - model.xDA[g,t] <= model.RR[g]
            
            self.model.ramp_down = pyo.Constraint(self.model.G, self.model.T, rule=ramp_rate_down)

            # Generation limits without commitment status
            def generation_limits(model, g, t):
                return model.xDA[g,t] <= model.CAP[g]
            
            self.model.generation_limits = pyo.Constraint(self.model.G, self.model.T, rule=generation_limits)

        # Add storage constraints only if storage exists
        if hasattr(self.model, 'e') and (len(self.model.B) > 0 or isinstance(self.model.B, pyo.RangeSet) or self.num_storage is None):
            # Storage energy balance
            def storage_balance(model, b, t):
                if t == 1:
                    return (model.e[b,t] == model.E0[b] + 
                            model.ETA_CH[b] * model.p_ch[b,t] - 
                            (1/model.ETA_DCH[b]) * model.p_dch[b,t])
                else:
                    return (model.e[b,t] == model.e[b,t-1] + 
                            model.ETA_CH[b] * model.p_ch[b,t] - 
                            (1/model.ETA_DCH[b]) * model.p_dch[b,t])

        # Record duals for market analysis
        self.model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)
        
    def create_instance(self, data):
        """
        Create a concrete instance of the model using provided data.
        
        Parameters:
        -----------
        data : dict
            Dictionary containing the data for the model parameters
            
        Returns:
        --------
        instance : ConcreteModel
            A concrete instance of the model
        """
        return self.model.create_instance(data)

### RT Simulation Model with Storage Integration

In [9]:
class RTSimModel:
    def __init__(self, num_periods=24, num_scenarios=5, num_generators=None, num_storage=None):
        """
        Initialize the RTSim model with flexible set dimensions.
        
        Parameters:
        -----------
        num_periods : int, optional
            Number of time periods (default: 24)
        num_scenarios : int, optional
            Number of scenarios (default: 5)
        num_generators : int, optional
            Number of generators (if None, determined from data)
        num_storage : int, optional
            Number of storage units (if None, determined from data)
        """
        self.num_periods = num_periods
        self.num_scenarios = num_scenarios
        self.num_generators = num_generators
        self.num_storage = num_storage
        
        self.model = pyo.AbstractModel()
        self._define_sets()
        self._define_parameters()
        self._define_variables()
        self._define_objective()
        self._define_constraints()
    
    def _define_sets(self):
        # Sets
        self.model.T = pyo.RangeSet(1, self.num_periods)    # Set of time periods
        self.model.S = pyo.RangeSet(1, self.num_scenarios)  # Set of scenarios
        
        # Generator set - can be determined from data
        if self.num_generators is not None and self.num_generators > 0:
            self.model.G = pyo.RangeSet(1, self.num_generators)
        else:
            self.model.G = pyo.Set()  # Will be populated when data is loaded or can be empty
        
        # Storage set - can be determined from data
        if self.num_storage is not None and self.num_storage > 0:
            self.model.B = pyo.RangeSet(1, self.num_storage)
        else:
            self.model.B = pyo.Set()  # Will be populated when data is loaded or can be empty
    
    def _define_parameters(self):
        # Original Parameters
        self.model.VC = pyo.Param(self.model.G)           # Variable cost
        self.model.VCUP = pyo.Param(self.model.G)         # Variable cost up
        self.model.VCDN = pyo.Param(self.model.G)         # Variable cost down
        self.model.CAP = pyo.Param(self.model.G)          # Generator capacity
        self.model.RR = pyo.Param(self.model.G)           # Ramp rate

        self.model.prob = pyo.Param(self.model.S)         # Scenario probability
        self.model.RE = pyo.Param(self.model.S, self.model.T)  # Renewable generation by scenario and time
        self.model.DEMAND = pyo.Param(self.model.T)       # Hourly demand
        self.model.D1 = pyo.Param(within=pyo.NonNegativeIntegers)  # Linear demand cost coefficient
        self.model.D2 = pyo.Param(within=pyo.NonNegativeIntegers)  # Quadratic demand cost coefficient
        self.model.xDA = pyo.Param(self.model.G, self.model.T) # DA schedule by generator and time
        self.model.REDA = pyo.Param(self.model.T)         # DA renewable schedule by time
        self.model.PEN = pyo.Param(within=pyo.NonNegativeIntegers)   # Upward penalty
        self.model.PENDN = pyo.Param()                    # Downward penalty
        self.model.DAdr = pyo.Param(self.model.T)         # DA demand response by time

        # Storage Parameters
        self.model.E_MAX = pyo.Param(self.model.B)        # Maximum energy capacity
        self.model.P_MAX = pyo.Param(self.model.B)        # Maximum power capacity
        self.model.ETA_CH = pyo.Param(self.model.B)       # Charging efficiency
        self.model.ETA_DCH = pyo.Param(self.model.B)      # Discharging efficiency
        self.model.E0 = pyo.Param(self.model.B)           # Initial state of charge
        self.model.STORAGE_COST = pyo.Param(self.model.B)  # Operating cost per MWh of throughput
        self.model.E_FINAL = pyo.Param(self.model.B)      # Required final state of charge
        
        # Storage DA parameters
        self.model.e_DA = pyo.Param(self.model.B, self.model.T)  # DA energy level
        self.model.p_ch_DA = pyo.Param(self.model.B, self.model.T)  # DA charging
        self.model.p_dch_DA = pyo.Param(self.model.B, self.model.T)  # DA discharging
    
    def _define_variables(self):
        # Variables that depend on generators
        if len(self.model.G) > 0 or isinstance(self.model.G, pyo.RangeSet) or self.num_generators is None:
            # RT adjustment variables for each scenario and time
            self.model.xup = pyo.Var(self.model.S, self.model.G, self.model.T, domain=pyo.NonNegativeReals)  # Generator up adjustment
            self.model.xdn = pyo.Var(self.model.S, self.model.G, self.model.T, domain=pyo.NonNegativeReals)  # Generator down adjustment
        
        # Variables independent of generators and storage
        self.model.d = pyo.Var(self.model.S, self.model.T)     # RT demand response
        self.model.rgup = pyo.Var(self.model.S, self.model.T, domain=pyo.NonNegativeReals)  # RE up adjustment
        self.model.rgdn = pyo.Var(self.model.S, self.model.T, domain=pyo.NonNegativeReals)  # RE down adjustment
        self.model.sdup = pyo.Var(self.model.S, self.model.T, domain=pyo.NonNegativeReals)  # Shortage
        self.model.sddn = pyo.Var(self.model.S, self.model.T, domain=pyo.NonNegativeReals)  # Surplus

        # Variables that depend on storage
        if len(self.model.B) > 0 or isinstance(self.model.B, pyo.RangeSet) or self.num_storage is None:
            # Storage Variables
            self.model.e = pyo.Var(self.model.S, self.model.B, self.model.T, domain=pyo.NonNegativeReals)     # Energy level
            self.model.p_ch = pyo.Var(self.model.S, self.model.B, self.model.T, domain=pyo.NonNegativeReals)  # Charging power
            self.model.p_dch = pyo.Var(self.model.S, self.model.B, self.model.T, domain=pyo.NonNegativeReals) # Discharging power
            self.model.b_up = pyo.Var(self.model.S, self.model.B, self.model.T, domain=pyo.NonNegativeReals)  # Storage up adjustment
            self.model.b_dn = pyo.Var(self.model.S, self.model.B, self.model.T, domain=pyo.NonNegativeReals)  # Storage down adjustment
    
    def _define_objective(self):
        # Objective Function
        def obj_expression(m):
            objective_terms = []
            
            for s in m.S:
                scenario_terms = []
                
                # Generator adjustment costs - only if generators exist
                if hasattr(m, 'xup') and len(m.G) > 0:
                    scenario_terms.append(
                        sum(m.VCUP[g] * m.xup[s,g,t] - m.VCDN[g] * m.xdn[s,g,t] for g in m.G for t in m.T)
                    )
                
                # Shortage/surplus penalties
                scenario_terms.append(m.PENDN * sum(m.sdup[s,t] for t in m.T))
                scenario_terms.append(m.PEN * sum(m.sddn[s,t] for t in m.T))
                
                # Demand response costs
                scenario_terms.append(
                    sum((m.D1 * (m.DAdr[t] + m.d[s,t]) + 
                         m.D2 * (m.DAdr[t] + m.d[s,t]) * (m.DAdr[t] + m.d[s,t])) for t in m.T)
                )
                scenario_terms.append(
                    -sum((m.D1 * m.DAdr[t] + m.D2 * m.DAdr[t] * m.DAdr[t]) for t in m.T)
                )
                
                # Storage operating costs - only if storage exists
                if hasattr(m, 'p_ch') and len(m.B) > 0:
                    scenario_terms.append(
                        sum(m.STORAGE_COST[b] * (m.p_ch[s,b,t] + m.p_dch[s,b,t]) for b in m.B for t in m.T)
                    )
                
                # Add the weighted scenario terms to the overall objective
                objective_terms.append(m.prob[s] * sum(scenario_terms))
            
            return sum(objective_terms)

        self.model.OBJ = pyo.Objective(rule=obj_expression)
    
    def _define_constraints(self):
        # RT energy balance for each scenario and time period
        def RT_energy_balance(model, s, t):
            gen_adjustment = 0
            storage_adjustment = 0
            
            # Generator adjustments - only if generators exist
            if hasattr(model, 'xup') and len(model.G) > 0:
                gen_adjustment = sum(model.xup[s,g,t] - model.xdn[s,g,t] for g in model.G)
            
            # Storage adjustments - only if storage exists
            if hasattr(model, 'p_ch') and len(model.B) > 0:
                storage_adjustment = sum(
                    model.p_dch[s,b,t] - model.p_ch[s,b,t] - 
                    model.p_dch_DA[b,t] + model.p_ch_DA[b,t] for b in model.B
                )
            
            return (gen_adjustment +
                    model.rgup[s,t] - model.rgdn[s,t] +
                    storage_adjustment +
                    model.d[s,t] == 0)
                    
        self.model.Con3 = pyo.Constraint(self.model.S, self.model.T, rule=RT_energy_balance)

        # RT renewable availability
        def RT_RE_availability(model, s, t):
            return (model.rgup[s,t] - model.rgdn[s,t] + model.sdup[s,t] - model.sddn[s,t] == 
                    model.RE[s,t] - model.REDA[t])
                    
        self.model.Con4 = pyo.Constraint(self.model.S, self.model.T, rule=RT_RE_availability)

        # Generator constraints - only if generators exist
        if hasattr(self.model, 'xup') and (len(self.model.G) > 0 or isinstance(self.model.G, pyo.RangeSet) or self.num_generators is None):
            # Generator ramping constraints
            def RT_ramp_up(model, s, g, t):
                return model.xup[s,g,t] <= model.RR[g]
                
            self.model.Con5up = pyo.Constraint(self.model.S, self.model.G, self.model.T, rule=RT_ramp_up)

            def RT_ramp_dn(model, s, g, t):
                return model.xdn[s,g,t] <= model.RR[g]
                
            self.model.Con5dn = pyo.Constraint(self.model.S, self.model.G, self.model.T, rule=RT_ramp_dn)

            # Generator capacity constraints
            def RT_capacity_cons(model, s, g, t):
                return model.xDA[g,t] + model.xup[s,g,t] <= model.CAP[g]
                
            self.model.Con6 = pyo.Constraint(self.model.S, self.model.G, self.model.T, rule=RT_capacity_cons)

            def RT_capacity_min(model, s, g, t):
                return model.xDA[g,t] - model.xdn[s,g,t] >= 0
                
            self.model.Con7 = pyo.Constraint(self.model.S, self.model.G, self.model.T, rule=RT_capacity_min)

        # Storage Constraints - only if storage exists
        if hasattr(self.model, 'e') and (len(self.model.B) > 0 or isinstance(self.model.B, pyo.RangeSet) or self.num_storage is None):
            # Storage energy balance
            def storage_balance(model, s, b, t):
                if t == 1:
                    return (model.e[s,b,t] == model.E0[b] + 
                            model.ETA_CH[b] * model.p_ch[s,b,t] - 
                            (1/model.ETA_DCH[b]) * model.p_dch[s,b,t])
                return (model.e[s,b,t] == model.e[s,b,t-1] + 
                        model.ETA_CH[b] * model.p_ch[s,b,t] - 
                        (1/model.ETA_DCH[b]) * model.p_dch[s,b,t])
                        
            self.model.storage_balance = pyo.Constraint(self.model.S, self.model.B, self.model.T, rule=storage_balance)

            # Storage capacity constraint
            def storage_capacity(model, s, b, t):
                return model.e[s,b,t] <= model.E_MAX[b]
                
            self.model.storage_capacity = pyo.Constraint(self.model.S, self.model.B, self.model.T, rule=storage_capacity)

            # Power limits
            def power_limits(model, s, b, t):
                return model.p_ch[s,b,t] + model.p_dch[s,b,t] <= model.P_MAX[b]
                
            self.model.power_limits = pyo.Constraint(self.model.S, self.model.B, self.model.T, rule=power_limits)
            
            # Storage adjustment limits
            def storage_adjustment_limits(model, s, b, t):
                return model.b_up[s,b,t] + model.b_dn[s,b,t] <= 0.5 * model.P_MAX[b]
                
            self.model.storage_adjustment_limits = pyo.Constraint(self.model.S, self.model.B, self.model.T, 
                                                                rule=storage_adjustment_limits)

            # Final state of charge requirement
            def final_soc(model, s, b):
                return model.e[s,b,self.num_periods] >= model.E_FINAL[b]
                
            self.model.final_soc = pyo.Constraint(self.model.S, self.model.B, rule=final_soc)

            # Storage ramping constraints
            def storage_ramp_rate(model, s, b, t):
                if t == 1:
                    return pyo.Constraint.Skip
                # This constraint with 'abs' needs to be reformulated for a solver
                # Here's a simple reformulation with two constraints
                return [
                    model.p_ch[s,b,t] - model.p_ch[s,b,t-1] <= 0.25 * model.P_MAX[b],
                    model.p_ch[s,b,t-1] - model.p_ch[s,b,t] <= 0.25 * model.P_MAX[b],
                    model.p_dch[s,b,t] - model.p_dch[s,b,t-1] <= 0.25 * model.P_MAX[b],
                    model.p_dch[s,b,t-1] - model.p_dch[s,b,t] <= 0.25 * model.P_MAX[b]
                ]
                        
            self.model.storage_ramp = pyo.Constraint(self.model.S, self.model.B, self.model.T, rule=storage_ramp_rate)

        # Record duals
        self.model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)
        
    def create_instance(self, data):
        """
        Create a concrete instance of the model using provided data.
        
        Parameters:
        -----------
        data : dict
            Dictionary containing the data for the model parameters
            
        Returns:
        --------
        instance : ConcreteModel
            A concrete instance of the model
        """
        return self.model.create_instance(data)

Data Processing

In [10]:
class DataProcessor:
    def __init__(self, csv_path):
        self.csv_path = csv_path
        self.df = None
        self.time_periods = range(1, 25)  # 24 hour periods
        self.hour_mapping = {
            '0800': 1, '0900': 2, '1000': 3, '1100': 4, '1200': 5, '1300': 6,
            '1400': 7, '1500': 8, '1600': 9, '1700': 10, '1800': 11, '1900': 12,
            '2000': 13, '2100': 14, '2200': 15, '2300': 16, '0000': 17, '0100': 18,
            '0200': 19, '0300': 20, '0400': 21, '0500': 22, '0600': 23, '0700': 24
        }

    def load_data(self):
        """Load and preprocess the CSV data"""
        try:
            self.df = pd.read_csv(self.csv_path)
            
            # Reorder columns to match time periods
            hour_cols = [col for col in self.df.columns if col in self.hour_mapping]
            ordered_cols = sorted(hour_cols, key=lambda x: self.hour_mapping[x])
            self.df_ordered = self.df[['Type', 'Index'] + ordered_cols] if 'Type' in self.df.columns and 'Index' in self.df.columns else self.df
            
            return self.df_ordered
        except Exception as e:
            print(f"Error loading data: {str(e)}")
            # Create a minimal dataframe if file doesn't exist or is invalid
            self.df = pd.DataFrame({'Type': ['Forecast', 'Actual'], 'Index': [1, 1]})
            for hour in self.hour_mapping.keys():
                self.df[hour] = 1.0  # Default values
            self.df_ordered = self.df
            return self.df_ordered

    def prepare_da_data(self, forecast_horizon=24):
        """Prepare data for DA model including storage parameters"""
        if self.df is None:
            self.load_data()

        # Filter for forecast data or use first row if not present
        if 'Type' in self.df.columns:
            da_data = self.df[self.df['Type'] == 'Forecast'].copy()
            if da_data.empty:
                da_data = self.df.iloc[[0]].copy()
        else:
            da_data = self.df.iloc[[0]].copy()
        
        # Create DA parameter dictionary
        da_params = {
            'DEMAND': {t: 1.0 for t in self.time_periods}  # Default values
        }
        
        # Try to populate with actual data if available
        try:
            for col, t in self.hour_mapping.items():
                if col in da_data.columns:
                    da_params['DEMAND'][t] = float(da_data.iloc[0][col])
        except Exception as e:
            print(f"Error preparing DA demand data: {str(e)}")
        
        da_params['REDA'] = {t: 0.0 for t in self.time_periods}  # Initialize with zeros
        
        # Add generator parameters
        da_params.update(self.generate_generator_params())
        
        # Add storage parameters
        da_params.update(self.generate_storage_params())
        
        # Add FO parameters
        da_params.update(self.generate_fo_params())
        
        return da_params

    def prepare_rt_data(self, num_scenarios=5):
        """Prepare data for RT model with scenario generation"""
        if self.df is None:
            self.load_data()

        # Generate synthetic scenarios if actual data not available
        scenarios = {}
        for s in range(1, num_scenarios + 1):
            scenarios[s] = {t: 1.0 for t in self.time_periods}  # Default values
            
        # Try to use actual data if available
        if 'Type' in self.df.columns:
            rt_data = self.df[self.df['Type'] == 'Actual'].copy()
            if not rt_data.empty:
                # Try to populate scenarios with actual data
                try:
                    for s in range(1, min(num_scenarios, len(rt_data)) + 1):
                        for col, t in self.hour_mapping.items():
                            if col in rt_data.columns and s <= len(rt_data):
                                scenarios[s][t] = float(rt_data.iloc[s-1][col])
                except Exception as e:
                    print(f"Error processing scenario data: {str(e)}")

        # Create RT parameter dictionary
        rt_params = {
            'RE': scenarios,
            'prob': {s: 1.0/num_scenarios for s in range(1, num_scenarios + 1)},
            'DEMAND': {t: 1.0 for t in self.time_periods}  # Default demand
        }
        
        # Try to populate RT demand from first scenario if available
        try:
            if 'Type' in self.df.columns:
                rt_data = self.df[self.df['Type'] == 'Actual'].copy()
                if not rt_data.empty:
                    for col, t in self.hour_mapping.items():
                        if col in rt_data.columns:
                            rt_params['DEMAND'][t] = float(rt_data.iloc[0][col])
        except Exception as e:
            print(f"Error preparing RT demand: {str(e)}")

        # Add generator parameters
        rt_params.update(self.generate_generator_params())
        
        # Add storage parameters
        rt_params.update(self.generate_storage_params())
        
        return rt_params

    def generate_generator_params(self, num_generators=5):
        """Generate default generator parameters"""
        generator_params = {
            'CAP': {g: 100.0 if g == 1 else 20.0 for g in range(1, num_generators + 1)},  # MW
            'VC': {g: 20.0 + 10.0 * (g-1) for g in range(1, num_generators + 1)},  # $/MWh
            'VCUP': {g: 30.0 + 15.0 * (g-1) for g in range(1, num_generators + 1)},  # $/MWh
            'VCDN': {g: 10.0 + 5.0 * (g-1) for g in range(1, num_generators + 1)},  # $/MWh
            'RR': {g: 20.0 if g == 1 else 10.0 for g in range(1, num_generators + 1)},  # MW/hour
            'MIN_UP_TIME': {g: 1 for g in range(1, num_generators + 1)},  # Hours
            'MIN_DOWN_TIME': {g: 1 for g in range(1, num_generators + 1)},  # Hours
            'STARTUP_COST': {g: 100.0 + 50.0 * (g-1) for g in range(1, num_generators + 1)},  # $
            'SHUTDOWN_COST': {g: 50.0 + 25.0 * (g-1) for g in range(1, num_generators + 1)},  # $
            'INITIAL_STATUS': {g: 1 for g in range(1, num_generators + 1)}  # All units initially on
        }
        return generator_params

    def generate_storage_params(self, num_storage_units=3):
        """Generate default storage parameters"""
        storage_params = {
            'E_MAX': {b: 1000.0 for b in range(1, num_storage_units + 1)},  # MWh
            'P_MAX': {b: 200.0 for b in range(1, num_storage_units + 1)},   # MW
            'ETA_CH': {b: 0.95 for b in range(1, num_storage_units + 1)},   # charging efficiency
            'ETA_DCH': {b: 0.95 for b in range(1, num_storage_units + 1)},  # discharging efficiency
            'E0': {b: 500.0 for b in range(1, num_storage_units + 1)},      # initial SOC
            'E_FINAL': {b: 500.0 for b in range(1, num_storage_units + 1)}, # final SOC requirement
            'STORAGE_COST': {b: 5.0 for b in range(1, num_storage_units + 1)} # $/MWh operating cost
        }
        return storage_params
    
    def generate_fo_params(self, num_tiers=4):
        """Generate default FO parameters"""
        fo_params = {
            'D1': {None: 10},  # Linear cost coefficient for demand slack
            'D2': {None: 0.1}, # Quadratic cost coefficient for demand slack
            'PEN': {None: 500}, # Penalty for inadequate flexibility up
            'PENDN': {None: 200}, # Penalty for inadequate flexibility down
            'smallM': {None: 0.0001}, # Parameter for alternative optima
            'probTU': {r: 0.2 + 0.1 * (r-1) for r in range(1, num_tiers + 1)}, # Probabilities of exercising FO up
            'probTD': {r: 0.2 + 0.1 * (r-1) for r in range(1, num_tiers + 1)}  # Probabilities of exercising FO down
        }
        return fo_params

Model Solver

In [11]:
class ModelSolver:
    def __init__(self, solver_path=None):
        """Initialize the model solver with optional solver path"""
        self.solver_path = solver_path
        self.opt = self._setup_solver()
        
    def _setup_solver(self):
        """Setup the solver using CPLEX"""
        try:
            if self.solver_path:
                opt = pyo.SolverFactory('cplex', executable=self.solver_path)    
            else:
                opt = pyo.SolverFactory('cplex')
                    
            if not opt.available():
                print("ERROR: CPLEX is unavailable. Please check installation.")
                return None
               
            return opt
        except Exception as e:
            print(f"Error setting up solver: {str(e)}")
            return None
    
    def solve_da_model(self, da_data):
        """Solve the DA model with Flexibility Options"""
        try:
            # Create DAFOModel instance
            dafo_model = DAFOModel()
            
            # Fix data structure to ensure it's compatible with Pyomo models
            fixed_data = self._fix_data_structure(da_data)
            
            # Create model instance
            instance = dafo_model.model.create_instance({None: fixed_data})
            print("DA model instance created successfully")
            
            # Solve model
            result = self.solve_with_handling(instance)
            
            if result:
                # Store key outputs
                outputs = self._extract_da_outputs(instance)
                return instance, outputs
            else:
                print("Failed to solve DA model")
                return None, None
                
        except Exception as e:
            print(f"Error in DA model: {str(e)}")
            return None, None
    
    def solve_rt_model(self, rt_data, da_outputs=None):
        """Solve the RT model for each scenario"""
        try:
            # Create RTSimModel instance
            rtsim_model = RTSimModel()
            
            # Fix data structure
            fixed_data = self._fix_data_structure(rt_data)
            
            # Add DA outputs to RT data if provided
            if da_outputs:
                for key, value in da_outputs.items():
                    if key not in fixed_data:
                        fixed_data[key] = value
            
            # Create model instance
            instance = rtsim_model.model.create_instance({None: fixed_data})
            print("RT model instance created successfully")
            
            # Solve model
            result = self.solve_with_handling(instance)
            
            if result:
                # Store key outputs
                outputs = self._extract_rt_outputs(instance)
                return instance, outputs
            else:
                print("Failed to solve RT model")
                return None, None
                
        except Exception as e:
            print(f"Error in RT model: {str(e)}")
            return None, None
    
    def solve_with_handling(self, model):
        """Solve a Pyomo model with error handling"""
        try:
            result = self.opt.solve(model, tee=True)
            
            if (result.solver.status == pyo.SolverStatus.ok and 
                result.solver.termination_condition == pyo.TerminationCondition.optimal):
                print("Model solved successfully")
                return True
            else:
                print(f"Solver status: {result.solver.status}")
                print(f"Termination condition: {result.solver.termination_condition}")
                return False
                
        except Exception as e:
            print(f"Error solving model: {str(e)}")
            return False
    
    def _fix_data_structure(self, data_dict):
        """Fix common issues with data structure for Pyomo models"""
        fixed_data = {}
        
        # Ensure all required parameters exist with proper structure
        for param in ['DEMAND', 'D1', 'D2', 'PEN', 'PENDN', 'smallM']:
            if param not in data_dict:
                # Add default values based on parameter type
                if param in ['D1', 'D2', 'PEN', 'PENDN']:
                    fixed_data[param] = {None: 10}  # Default penalty values as mapping
                elif param == 'smallM':
                    fixed_data[param] = {None: 0.0001}  # Default small value for smallM
                else:
                    fixed_data[param] = {t: 1.0 for t in range(1, 25)}  # Default hourly values
        
        # Process each parameter in the input data
        for param, value in data_dict.items():
            # Skip None values
            if value is None:
                continue
                
            # Handle scalar parameters - convert to mapping type
            if param in ['D1', 'D2', 'PEN', 'PENDN', 'smallM']:
                fixed_data[param] = {None: float(value) if isinstance(value, (int, float, str)) else 
                                        (10.0 if param != 'smallM' else 0.0001)}
            
            # Handle indexed parameters
            elif isinstance(value, dict):
                # Ensure the dict keys are correct format
                fixed_data[param] = {
                    int(k) if isinstance(k, str) and k.isdigit() else k: v 
                    for k, v in value.items()
                }
            
            # Handle other cases - convert to mapping type
            else:
                fixed_data[param] = {None: value}
        
        # Ensure RE parameter has correct structure (scenario, time)
        if 'RE' not in fixed_data:
            fixed_data['RE'] = {(s, t): 0.0 for s in range(1, 6) for t in range(1, 25)}
            
        return fixed_data
    
    def _extract_da_outputs(self, instance):
        """Extract key outputs from DA model instance"""
        outputs = {}
        
        try:
            # Extract energy schedules
            outputs['xDA'] = {(g, t): instance.xDA[g, t].value for g in instance.G for t in instance.T}
            
            # Extract renewable schedule
            outputs['REDA'] = {t: instance.rgDA[t].value for t in instance.T}
            
            # Extract storage schedules
            outputs['e_DA'] = {(b, t): instance.e[b, t].value for b in instance.B for t in instance.T}
            outputs['p_ch_DA'] = {(b, t): instance.p_ch[b, t].value for b in instance.B for t in instance.T}
            outputs['p_dch_DA'] = {(b, t): instance.p_dch[b, t].value for b in instance.B for t in instance.T}
            
            # Extract FO schedules for all suppliers (generators + storage)
            outputs['hsu'] = {(r, g, t): instance.hsu[r, g, t].value for r in instance.R for g in instance.G for t in instance.T}
            outputs['hsd'] = {(r, g, t): instance.hsd[r, g, t].value for r in instance.R for g in instance.G for t in instance.T}
            outputs['bsu'] = {(r, b, t): instance.bsu[r, b, t].value for r in instance.R for b in instance.B for t in instance.T}
            outputs['bsd'] = {(r, b, t): instance.bsd[r, b, t].value for r in instance.R for b in instance.B for t in instance.T}
            
            # Extract FO demand
            outputs['hdu'] = {(r, t): instance.hdu[r, t].value for r in instance.R for t in instance.T}
            outputs['hdd'] = {(r, t): instance.hdd[r, t].value for r in instance.R for t in instance.T}
            
            # Extract prices
            outputs['DA_price'] = {t: instance.dual[instance.Con3[t]] for t in instance.T}
            outputs['FO_up_price'] = {(r, t): instance.dual[instance.Con4UP[r, t]] for r in instance.R for t in instance.T}
            outputs['FO_down_price'] = {(r, t): instance.dual[instance.Con4DN[r, t]] for r in instance.R for t in instance.T}
            
            # Extract total costs
            outputs['total_cost'] = instance.OBJ()
            
            # Extract commitment status
            outputs['u'] = {(g, t): instance.u[g, t].value for g in instance.G for t in instance.T}
            
            # Extract demand slack
            outputs['DAdr'] = {t: instance.d[t].value for t in instance.T}
            
        except Exception as e:
            print(f"Error extracting DA outputs: {str(e)}")
        
        return outputs
    
    def _extract_rt_outputs(self, instance):
        """Extract key outputs from RT model instance"""
        outputs = {}
        
        try:
            # Extract RT energy adjustments
            outputs['xup'] = {(s, g, t): instance.xup[s, g, t].value for s in instance.S for g in instance.G for t in instance.T}
            outputs['xdn'] = {(s, g, t): instance.xdn[s, g, t].value for s in instance.S for g in instance.G for t in instance.T}
            
            # Extract storage adjustments
            outputs['e_RT'] = {(s, b, t): instance.e[s, b, t].value for s in instance.S for b in instance.B for t in instance.T}
            outputs['p_ch_RT'] = {(s, b, t): instance.p_ch[s, b, t].value for s in instance.S for b in instance.B for t in instance.T}
            outputs['p_dch_RT'] = {(s, b, t): instance.p_dch[s, b, t].value for s in instance.S for b in instance.B for t in instance.T}
            outputs['b_up'] = {(s, b, t): instance.b_up[s, b, t].value for s in instance.S for b in instance.B for t in instance.T}
            outputs['b_dn'] = {(s, b, t): instance.b_dn[s, b, t].value for s in instance.S for b in instance.B for t in instance.T}
            
            # Extract RE adjustments
            outputs['rgup'] = {(s, t): instance.rgup[s, t].value for s in instance.S for t in instance.T}
            outputs['rgdn'] = {(s, t): instance.rgdn[s, t].value for s in instance.S for t in instance.T}
            
            # Extract shortage/surplus
            outputs['sdup'] = {(s, t): instance.sdup[s, t].value for s in instance.S for t in instance.T}
            outputs['sddn'] = {(s, t): instance.sddn[s, t].value for s in instance.S for t in instance.T}
            
            # Extract RT prices
            outputs['RT_price'] = {(s, t): instance.dual[instance.Con3[s, t]] for s in instance.S for t in instance.T}
            
            # Extract total costs per scenario
            outputs['scenario_costs'] = {s: sum(
                instance.prob[s] * (
                    sum(instance.VCUP[g] * instance.xup[s, g, t].value - 
                        instance.VCDN[g] * instance.xdn[s, g, t].value 
                        for g in instance.G for t in instance.T)
                )) for s in instance.S}
            
        except Exception as e:
            print(f"Error extracting RT outputs: {str(e)}")
        
        return outputs


class ResultsAnalyzer:
    """Class for analyzing and visualizing results"""
    
    def __init__(self, da_outputs, rt_outputs):
        """Initialize with outputs from DA and RT models"""
        self.da_outputs = da_outputs
        self.rt_outputs = rt_outputs
        
    def hourly_analysis(self):
        """Perform hourly analysis of results"""
        # Create DataFrame for hourly results
        hourly_results = pd.DataFrame(index=range(1, 25))
        
        # Add DA energy prices
        if 'DA_price' in self.da_outputs:
            hourly_results['DA_price'] = pd.Series(self.da_outputs['DA_price'])
        
        # Add RT energy prices (average across scenarios)
        if 'RT_price' in self.rt_outputs:
            rt_prices = pd.DataFrame.from_dict({t: [self.rt_outputs['RT_price'].get((s, t), 0) 
                                                  for s in range(1, 6)] 
                                            for t in range(1, 25)}, 
                                           orient='index')
            hourly_results['RT_price_mean'] = rt_prices.mean(axis=1)
            hourly_results['RT_price_min'] = rt_prices.min(axis=1)
            hourly_results['RT_price_max'] = rt_prices.max(axis=1)
        
        # Add generator schedules
        for g in range(1, 6):
            g_schedule = pd.Series({t: self.da_outputs['xDA'].get((g, t), 0) for t in range(1, 25)})
            hourly_results[f'Gen_{g}_DA'] = g_schedule
        
        # Add storage schedules
        for b in range(1, 4):
            b_charge = pd.Series({t: self.da_outputs['p_ch_DA'].get((b, t), 0) for t in range(1, 25)})
            b_discharge = pd.Series({t: self.da_outputs['p_dch_DA'].get((b, t), 0) for t in range(1, 25)})
            b_energy = pd.Series({t: self.da_outputs['e_DA'].get((b, t), 0) for t in range(1, 25)})
            
            hourly_results[f'Storage_{b}_charge'] = b_charge
            hourly_results[f'Storage_{b}_discharge'] = b_discharge
            hourly_results[f'Storage_{b}_energy'] = b_energy
        
        # Add FO volumes by tier
        for r in range(1, 5):
            # Sum FO up/down volumes across all suppliers for each tier
            fo_up = pd.Series({t: sum(self.da_outputs['hsu'].get((r, g, t), 0) for g in range(1, 6)) + 
                                  sum(self.da_outputs['bsu'].get((r, b, t), 0) for b in range(1, 4))
                               for t in range(1, 25)})
            
            fo_down = pd.Series({t: sum(self.da_outputs['hsd'].get((r, g, t), 0) for g in range(1, 6)) + 
                                    sum(self.da_outputs['bsd'].get((r, b, t), 0) for b in range(1, 4))
                                 for t in range(1, 25)})
            
            hourly_results[f'FO_up_tier_{r}'] = fo_up
            hourly_results[f'FO_down_tier_{r}'] = fo_down
        
        return hourly_results
    
    def plot_hourly_results(self, hourly_results):
        """Plot key hourly results"""
        fig, axes = plt.subplots(3, 1, figsize=(12, 15))
        
        # Plot 1: Energy Prices
        price_cols = [col for col in hourly_results.columns if 'price' in col.lower()]
        hourly_results[price_cols].plot(ax=axes[0])
        axes[0].set_title('Hourly Energy Prices')
        axes[0].set_xlabel('Hour')
        axes[0].set_ylabel('Price ($/MWh)')
        axes[0].grid(True)
        
        # Plot 2: Generator and Storage Schedules
        gen_cols = [col for col in hourly_results.columns if 'Gen_' in col]
        storage_discharge = [col for col in hourly_results.columns if 'discharge' in col]
        storage_charge = [col for col in hourly_results.columns if 'charge' in col]
        
        hourly_results[gen_cols + storage_discharge].plot(ax=axes[1])
        # Plot storage charging as negative values
        (-1 * hourly_results[storage_charge]).plot(ax=axes[1], linestyle='--')
        
        axes[1].set_title('Hourly Generation and Storage Schedules')
        axes[1].set_xlabel('Hour')
        axes[1].set_ylabel('Power (MW)')
        axes[1].grid(True)
        
        # Plot 3: FO Volumes
        fo_up_cols = [col for col in hourly_results.columns if 'FO_up_tier' in col]
        fo_down_cols = [col for col in hourly_results.columns if 'FO_down_tier' in col]
        
        hourly_results[fo_up_cols].plot(ax=axes[2])
        (-1 * hourly_results[fo_down_cols]).plot(ax=axes[2], linestyle='--')
        
        axes[2].set_title('Hourly Flexibility Option Volumes')
        axes[2].set_xlabel('Hour')
        axes[2].set_ylabel('FO Volume (MW)')
        axes[2].grid(True)
        
        plt.tight_layout()
        plt.savefig('hourly_results.png')
        plt.show()
        
    def calculate_revenues_costs(self):
        """Calculate revenues and costs for all participants"""
        results = {}
        
        # Initialize dictionaries for each participant type
        generator_results = {g: {} for g in range(1, 6)}
        storage_results = {b: {} for b in range(1, 4)}
        
        try:
            # Calculate DA energy revenues/costs for generators
            for g in range(1, 6):
                # DA energy revenues
                da_energy_revenue = sum(self.da_outputs['DA_price'][t] * self.da_outputs['xDA'].get((g, t), 0) 
                                        for t in range(1, 25))
                
                # DA FO premiums
                da_fo_up_revenue = sum(
                    self.da_outputs['FO_up_price'].get((r, t), 0) * self.da_outputs['hsu'].get((r, g, t), 0)
                    for r in range(1, 5) for t in range(1, 25)
                )
                
                da_fo_down_revenue = sum(
                    self.da_outputs['FO_down_price'].get((r, t), 0) * self.da_outputs['hsd'].get((r, g, t), 0)
                    for r in range(1, 5) for t in range(1, 25)
                )
                
                # RT adjustments
                rt_revenue = {}
                for s in range(1, 6):
                    rt_revenue[s] = sum(
                        self.rt_outputs['RT_price'].get((s, t), 0) * (
                            self.rt_outputs['xup'].get((s, g, t), 0) - self.rt_outputs['xdn'].get((s, g, t), 0)
                        ) for t in range(1, 25)
                    )
                
                # Total revenues
                generator_results[g]['DA_energy_revenue'] = da_energy_revenue
                generator_results[g]['DA_FO_up_revenue'] = da_fo_up_revenue
                generator_results[g]['DA_FO_down_revenue'] = da_fo_down_revenue
                generator_results[g]['RT_revenue'] = rt_revenue
                generator_results[g]['Total_revenue'] = da_energy_revenue + da_fo_up_revenue + da_fo_down_revenue + sum(rt_revenue.values()) / len(rt_revenue)
            
            # Calculate DA energy revenues/costs for storage units
            for b in range(1, 4):
                # DA energy revenues (discharge - charge)
                da_energy_revenue = sum(
                    self.da_outputs['DA_price'][t] * (
                        self.da_outputs['p_dch_DA'].get((b, t), 0) - self.da_outputs['p_ch_DA'].get((b, t), 0)
                    ) for t in range(1, 25)
                )
                
                # DA FO premiums
                da_fo_up_revenue = sum(
                    self.da_outputs['FO_up_price'].get((r, t), 0) * self.da_outputs['bsu'].get((r, b, t), 0)
                    for r in range(1, 5) for t in range(1, 25)
                )
                
                da_fo_down_revenue = sum(
                    self.da_outputs['FO_down_price'].get((r, t), 0) * self.da_outputs['bsd'].get((r, b, t), 0)
                    for r in range(1, 5) for t in range(1, 25)
                )
                
                # RT adjustments
                rt_revenue = {}
                for s in range(1, 6):
                    rt_revenue[s] = sum(
                        self.rt_outputs['RT_price'].get((s, t), 0) * (
                            self.rt_outputs['p_dch_RT'].get((s, b, t), 0) - self.rt_outputs['p_ch_RT'].get((s, b, t), 0)
                        ) for t in range(1, 25)
                    )
                
                # Storage operation costs
                storage_cost = sum(
                    self.da_outputs.get('STORAGE_COST', {}).get(b, 5.0) * (
                        self.da_outputs['p_ch_DA'].get((b, t), 0) + self.da_outputs['p_dch_DA'].get((b, t), 0)
                    ) for t in range(1, 25)
                )
                
                # Total revenues
                storage_results[b]['DA_energy_revenue'] = da_energy_revenue
                storage_results[b]['DA_FO_up_revenue'] = da_fo_up_revenue
                storage_results[b]['DA_FO_down_revenue'] = da_fo_down_revenue
                storage_results[b]['RT_revenue'] = rt_revenue
                storage_results[b]['Storage_cost'] = storage_cost
                storage_results[b]['Total_revenue'] = da_energy_revenue + da_fo_up_revenue + da_fo_down_revenue + sum(rt_revenue.values()) / len(rt_revenue) - storage_cost
            
            # Combine results
            results['generators'] = generator_results
            results['storage'] = storage_results
            
            # Calculate system costs
            system_costs = {
                'DA_cost': self.da_outputs.get('total_cost', 0),
                'RT_cost': sum(self.rt_outputs.get('scenario_costs', {}).values()),
                'Total_cost': self.da_outputs.get('total_cost', 0) + sum(self.rt_outputs.get('scenario_costs', {}).values())
            }
            results['system'] = system_costs
            
        except Exception as e:
            print(f"Error calculating revenues and costs: {str(e)}")
        
        return results
    
    def summarize_fo_effectiveness(self):
        """Analyze and summarize the effectiveness of FOs"""
        summary = {}
        
        try:
            # Calculate total FO volumes by hour and tier
            fo_volumes = pd.DataFrame(index=range(1, 25))
            
            # Sum up FO volumes from generators and storage
            for r in range(1, 5):
                up_vol = []
                down_vol = []
                
                for t in range(1, 25):
                    # Generator FO volumes
                    gen_up = sum(self.da_outputs['hsu'].get((r, g, t), 0) for g in range(1, 6))
                    gen_down = sum(self.da_outputs['hsd'].get((r, g, t), 0) for g in range(1, 6))
                    
                    # Storage FO volumes
                    storage_up = sum(self.da_outputs['bsu'].get((r, b, t), 0) for b in range(1, 4))
                    storage_down = sum(self.da_outputs['bsd'].get((r, b, t), 0) for b in range(1, 4))
                    
                    up_vol.append(gen_up + storage_up)
                    down_vol.append(gen_down + storage_down)
                
                fo_volumes[f'Up_Tier_{r}'] = up_vol
                fo_volumes[f'Down_Tier_{r}'] = down_vol
            
            # Calculate average FO prices by tier
            fo_prices = {}
            for r in range(1, 5):
                up_price = np.mean([self.da_outputs['FO_up_price'].get((r, t), 0) for t in range(1, 25)])
                down_price = np.mean([self.da_outputs['FO_down_price'].get((r, t), 0) for t in range(1, 25)])
                
                fo_prices[f'Up_Tier_{r}'] = up_price
                fo_prices[f'Down_Tier_{r}'] = down_price
            
            # Calculate price convergence between DA and RT markets
            da_prices = pd.Series(self.da_outputs.get('DA_price', {}))
            rt_avg_prices = pd.Series({
                t: np.mean([self.rt_outputs['RT_price'].get((s, t), 0) for s in range(1, 6)])
                for t in range(1, 25)
            })
            
            price_diff = (da_prices - rt_avg_prices).abs()
            price_convergence = {
                'mean_abs_diff': price_diff.mean(),
                'max_abs_diff': price_diff.max(),
                'price_diff_by_hour': price_diff.to_dict()
            }
            
            # Calculate FO exercise rates in RT
            fo_exercise = {}
            for r in range(1, 5):
                up_exercise = {}
                down_exercise = {}
                
                for t in range(1, 25):
                    up_vol = sum(self.da_outputs['hsu'].get((r, g, t), 0) for g in range(1, 6)) + \
                             sum(self.da_outputs['bsu'].get((r, b, t), 0) for b in range(1, 4))
                    
                    down_vol = sum(self.da_outputs['hsd'].get((r, g, t), 0) for g in range(1, 6)) + \
                               sum(self.da_outputs['bsd'].get((r, b, t), 0) for b in range(1, 4))
                    
                    # Calculate RT exercise counts
                    up_count = 0
                    down_count = 0
                    
                    for s in range(1, 6):
                        # Check if FO is exercised in scenario s
                        # For up FO: RT price > strike price and scenario <= tier
                        # For down FO: RT price < strike price and scenario > tier
                        
                        if s <= r and up_vol > 0:
                            up_count += 1
                        
                        if s > r and down_vol > 0:
                            down_count += 1
                    
                    if up_vol > 0:
                        up_exercise[t] = up_count / 5  # Normalize by scenario count
                    
                    if down_vol > 0:
                        down_exercise[t] = down_count / 5  # Normalize by scenario count
                
                fo_exercise[f'Up_Tier_{r}'] = up_exercise
                fo_exercise[f'Down_Tier_{r}'] = down_exercise
            
            # Combine results
            summary['fo_volumes'] = fo_volumes
            summary['fo_prices'] = fo_prices
            summary['price_convergence'] = price_convergence
            summary['fo_exercise'] = fo_exercise
            
            # Calculate storage contribution to FO
            storage_fo_contribution = {}
            for r in range(1, 5):
                storage_up = sum(sum(self.da_outputs['bsu'].get((r, b, t), 0) for b in range(1, 4)) 
                                 for t in range(1, 25))
                
                total_up = storage_up + sum(sum(self.da_outputs['hsu'].get((r, g, t), 0) for g in range(1, 6)) 
                                           for t in range(1, 25))
                
                storage_down = sum(sum(self.da_outputs['bsd'].get((r, b, t), 0) for b in range(1, 4)) 
                                   for t in range(1, 25))
                
                total_down = storage_down + sum(sum(self.da_outputs['hsd'].get((r, g, t), 0) for g in range(1, 6)) 
                                               for t in range(1, 25))
                
                if total_up > 0:
                    storage_fo_contribution[f'Up_Tier_{r}'] = storage_up / total_up
                else:
                    storage_fo_contribution[f'Up_Tier_{r}'] = 0
                
                if total_down > 0:
                    storage_fo_contribution[f'Down_Tier_{r}'] = storage_down / total_down
                else:
                    storage_fo_contribution[f'Down_Tier_{r}'] = 0
            
            summary['storage_fo_contribution'] = storage_fo_contribution
            
        except Exception as e:
            print(f"Error summarizing FO effectiveness: {str(e)}")
        
        return summary
    
    def plot_fo_volumes_prices(self, fo_summary):
        """Plot FO volumes and prices by tier"""
        if not fo_summary or 'fo_volumes' not in fo_summary or 'fo_prices' not in fo_summary:
            print("Insufficient data for plotting")
            return
        
        # Get FO volumes
        fo_volumes = fo_summary['fo_volumes']
        fo_prices = fo_summary['fo_prices']
        
        # Create figure
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        
        # Plot Up FO volumes
        up_cols = [col for col in fo_volumes.columns if 'Up_' in col]
        fo_volumes[up_cols].plot(ax=axes[0, 0], marker='o')
        axes[0, 0].set_title('Upward FO Volumes by Tier')
        axes[0, 0].set_xlabel('Hour')
        axes[0, 0].set_ylabel('FO Volume (MW)')
        axes[0, 0].grid(True)
        
        # Plot Down FO volumes
        down_cols = [col for col in fo_volumes.columns if 'Down_' in col]
        fo_volumes[down_cols].plot(ax=axes[0, 1], marker='o')
        axes[0, 1].set_title('Downward FO Volumes by Tier')
        axes[0, 1].set_xlabel('Hour')
        axes[0, 1].set_ylabel('FO Volume (MW)')
        axes[0, 1].grid(True)
        
        # Plot Up FO prices
        up_prices = pd.Series({int(k.split('_')[-1]): v for k, v in fo_prices.items() if 'Up_' in k})
        up_prices.plot(kind='bar', ax=axes[1, 0])
        axes[1, 0].set_title('Upward FO Prices by Tier')
        axes[1, 0].set_xlabel('Tier')
        axes[1, 0].set_ylabel('FO Price ($/MW)')
        axes[1, 0].grid(True)
        
        # Plot Down FO prices
        down_prices = pd.Series({int(k.split('_')[-1]): v for k, v in fo_prices.items() if 'Down_' in k})
        down_prices.plot(kind='bar', ax=axes[1, 1])
        axes[1, 1].set_title('Downward FO Prices by Tier')
        axes[1, 1].set_xlabel('Tier')
        axes[1, 1].set_ylabel('FO Price ($/MW)')
        axes[1, 1].grid(True)
        
        plt.tight_layout()
        plt.savefig('fo_volumes_prices.png')
        plt.show()
        
    def plot_storage_contribution(self, fo_summary):
        """Plot storage contribution to FO by tier"""
        if not fo_summary or 'storage_fo_contribution' not in fo_summary:
            print("Insufficient data for plotting")
            return
        
        # Get storage contribution data
        storage_contrib = fo_summary['storage_fo_contribution']
        
        # Create figure
        fig, ax = plt.subplots(figsize=(10, 6))
        
        # Prepare data for plotting
        tiers = sorted(list(set([int(k.split('_')[-1]) for k in storage_contrib.keys()])))
        up_contrib = [storage_contrib.get(f'Up_Tier_{r}', 0) * 100 for r in tiers]
        down_contrib = [storage_contrib.get(f'Down_Tier_{r}', 0) * 100 for r in tiers]
        
        # Create bar plot
        x = np.arange(len(tiers))
        width = 0.35
        
        ax.bar(x - width/2, up_contrib, width, label='Upward FO')
        ax.bar(x + width/2, down_contrib, width, label='Downward FO')
        
        ax.set_xlabel('Tier')
        ax.set_ylabel('Storage Contribution (%)')
        ax.set_title('Storage Contribution to Flexibility Options by Tier')
        ax.set_xticks(x)
        ax.set_xticklabels([f'Tier {r}' for r in tiers])
        ax.legend()
        ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig('storage_contribution.png')
        plt.show()
        
    def export_results(self, filename='fo_results.xlsx'):
        """Export results to Excel file"""
        try:
            with pd.ExcelWriter(filename) as writer:
                # Export hourly results
                hourly_results = self.hourly_analysis()
                hourly_results.to_excel(writer, 'Hourly_Results')
                
                # Export revenue/cost results
                rev_cost = self.calculate_revenues_costs()
                
                # Generator revenues
                gen_rev = pd.DataFrame()
                for g, values in rev_cost.get('generators', {}).items():
                    for key, value in values.items():
                        if key != 'RT_revenue':
                            gen_rev.loc[f'Generator_{g}', key] = value
                        else:
                            for s, rev in value.items():
                                gen_rev.loc[f'Generator_{g}', f'RT_revenue_sc{s}'] = rev
                
                gen_rev.to_excel(writer, 'Generator_Revenues')
                
                # Storage revenues
                storage_rev = pd.DataFrame()
                for b, values in rev_cost.get('storage', {}).items():
                    for key, value in values.items():
                        if key != 'RT_revenue':
                            storage_rev.loc[f'Storage_{b}', key] = value
                        else:
                            for s, rev in value.items():
                                storage_rev.loc[f'Storage_{b}', f'RT_revenue_sc{s}'] = rev
                
                storage_rev.to_excel(writer, 'Storage_Revenues')
                
                # System costs
                system_costs = pd.Series(rev_cost.get('system', {}))
                system_costs.to_frame('Value').to_excel(writer, 'System_Costs')
                
                # FO effectiveness summary
                fo_summary = self.summarize_fo_effectiveness()
                
                # FO volumes
                if 'fo_volumes' in fo_summary:
                    fo_summary['fo_volumes'].to_excel(writer, 'FO_Volumes')
                
                # FO prices
                if 'fo_prices' in fo_summary:
                    pd.Series(fo_summary['fo_prices']).to_frame('Value').to_excel(writer, 'FO_Prices')
                
                # Price convergence
                if 'price_convergence' in fo_summary:
                    price_conv = fo_summary['price_convergence']
                    pd.Series({
                        'Mean_Abs_Diff': price_conv.get('mean_abs_diff', 0),
                        'Max_Abs_Diff': price_conv.get('max_abs_diff', 0)
                    }).to_frame('Value').to_excel(writer, 'Price_Convergence')
                    
                    pd.Series(price_conv.get('price_diff_by_hour', {})).to_frame('Value').to_excel(writer, 'Price_Diff_By_Hour')
                
                # Storage contribution
                if 'storage_fo_contribution' in fo_summary:
                    pd.Series(fo_summary['storage_fo_contribution']).to_frame('Value').to_excel(writer, 'Storage_Contribution')
                
            print(f"Results exported to {filename}")
            
        except Exception as e:
            print(f"Error exporting results: {str(e)}")
    
    def run_full_analysis(self):
        """Run complete analysis and generate all plots"""
        # Perform hourly analysis
        hourly_results = self.hourly_analysis()
        self.plot_hourly_results(hourly_results)
        
        # Calculate revenues and costs
        rev_cost = self.calculate_revenues_costs()
        
        # Analyze FO effectiveness
        fo_summary = self.summarize_fo_effectiveness()
        self.plot_fo_volumes_prices(fo_summary)
        self.plot_storage_contribution(fo_summary)
        
        # Export results
        self.export_results()
        
        return {
            'hourly_results': hourly_results,
            'rev_cost': rev_cost,
            'fo_summary': fo_summary
        }

In [12]:
input_file = '2.csv'
output_dir = 'Flexibility_Options_Results'
solver_path = 'C:/Program Files/IBM/ILOG/CPLEX_Studio_Community2212/cplex/bin/x64_win64/cplex'

os.makedirs(output_dir, exist_ok=True)

def main():
    """Main function to execute the full workflow"""
    try:
        print("Starting Flexibility Options with Storage Analysis...")
        
        # Initialize data processor
        processor = DataProcessor(input_file)
        print("Data processor initialized.")
        
        # Prepare DA and RT data
        da_data = processor.prepare_da_data()
        rt_data = processor.prepare_rt_data()
        print("Data preparation completed.")
        
        # Initialize model solver
        solver = ModelSolver(solver_path)
        if solver.opt is None:
            print("ERROR: Solver initialization failed. Exiting...")
            return
        print("Model solver initialized.")
        
        # Solve DA model
        print("\nSolving DA model with Flexibility Options...")
        da_instance, da_outputs = solver.solve_da_model(da_data)
        
        if da_instance is None or da_outputs is None:
            print("ERROR: DA model solution failed. Exiting...")
            return
        print("DA model solved successfully.")
        
        # Solve RT model
        print("\nSolving RT model for each scenario...")
        rt_instance, rt_outputs = solver.solve_rt_model(rt_data, da_outputs)
        
        if rt_instance is None or rt_outputs is None:
            print("ERROR: RT model solution failed. Exiting...")
            return
        print("RT model solved successfully.")
        
        # Analyze results
        print("\nAnalyzing results...")
        analyzer = ResultsAnalyzer(da_outputs, rt_outputs)
        analysis_results = analyzer.run_full_analysis()
        print("Analysis completed and visualizations generated.")
        
        # Export key model outputs for further analysis
        export_model_outputs(da_instance, rt_instance, os.path.join(output_dir, 'model_outputs.xlsx'))
        print(f"Model outputs exported to {os.path.join(output_dir, 'model_outputs.xlsx')}")
        
        print("\nSummary:")
        print(f"- Total DA cost: ${da_outputs.get('total_cost', 0):.2f}")
        
        if analysis_results and 'fo_summary' in analysis_results and 'storage_fo_contribution' in analysis_results['fo_summary']:
            storage_contrib = analysis_results['fo_summary']['storage_fo_contribution']
            avg_storage_contrib = np.mean([v for k, v in storage_contrib.items()]) * 100
            print(f"- Average storage contribution to FOs: {avg_storage_contrib:.2f}%")
        
        print(f"- Full results available in: {output_dir}")
        
    except Exception as e:
        print(f"ERROR: {str(e)}")

def export_model_outputs(da_instance, rt_instance, filename):
    """Export raw model outputs for further analysis"""
    try:
        with pd.ExcelWriter(filename) as writer:
            # Export dual variables from DA model
            da_duals = {}
            for c in da_instance.component_objects(pyo.Constraint, active=True):
                for idx in c:
                    if idx in da_instance.dual:
                        da_duals[f"{c.name}_{idx}"] = da_instance.dual[idx]
            
            pd.Series(da_duals).to_frame('Value').to_excel(writer, 'DA_Duals')
            
            # Export storage-related variables from DA model
            storage_vars = {}
            for v_name in ['e', 'p_ch', 'p_dch', 'bsu', 'bsd']:
                if hasattr(da_instance, v_name):
                    v = getattr(da_instance, v_name)
                    for idx in v:
                        storage_vars[f"{v_name}_{idx}"] = v[idx].value
            
            pd.Series(storage_vars).to_frame('Value').to_excel(writer, 'DA_Storage_Variables')
            
            # Export RT energy prices
            if hasattr(rt_instance, 'dual'):
                rt_prices = {}
                for s in rt_instance.S:
                    for t in rt_instance.T:
                        if hasattr(rt_instance, 'Con3') and (s, t) in rt_instance.Con3:
                            if (s, t) in rt_instance.dual:
                                rt_prices[f"RT_price_s{s}_t{t}"] = rt_instance.dual[(s, t)]
                
                pd.Series(rt_prices).to_frame('Value').to_excel(writer, 'RT_Prices')
                
    except Exception as e:
        print(f"Error exporting model outputs: {str(e)}")


if __name__ == "__main__":
    main()

Starting Flexibility Options with Storage Analysis...
Data processor initialized.
Data preparation completed.
Model solver initialized.

Solving DA model with Flexibility Options...
ERROR: Constructing component 'VC' from data={1: 20.0, 2: 30.0, 3: 40.0, 4:
50.0, 5: 60.0} failed:
        RuntimeError: Failed to set value for param=VC, index=1, value=20.0.
    	source error message="Index '1' is not valid for indexed component 'VC'"
Error in DA model: Failed to set value for param=VC, index=1, value=20.0.
	source error message="Index '1' is not valid for indexed component 'VC'"
ERROR: DA model solution failed. Exiting...


In [13]:
import matplotlib.pyplot as plt

def plot_summary_results(summary_file):
    try:
        # Read the Excel file
        system_cost = pd.read_excel(summary_file, sheet_name='System_Cost', index_col=0)
        price_conv = pd.read_excel(summary_file, sheet_name='Price_Convergence', index_col=0)
        premium_up = pd.read_excel(summary_file, sheet_name='Premium_Up', index_col=0)
        premium_down = pd.read_excel(summary_file, sheet_name='Premium_Down', index_col=0)
        demand_cost = pd.read_excel(summary_file, sheet_name='Demand_Cost', index_col=0)
        curtail_cost = pd.read_excel(summary_file, sheet_name='Curtailment_Cost', index_col=0)
        
        # Check if dataframes contain numeric data
        if (system_cost.empty or price_conv.empty or premium_up.empty or 
            premium_down.empty or demand_cost.empty or curtail_cost.empty):
            print("Error: One or more sheets contain no data")
            return
            
        # Convert data to numeric, replacing non-numeric values with NaN
        system_cost = system_cost.apply(pd.to_numeric, errors='coerce')
        price_conv = price_conv.apply(pd.to_numeric, errors='coerce')
        premium_up = premium_up.apply(pd.to_numeric, errors='coerce')
        premium_down = premium_down.apply(pd.to_numeric, errors='coerce')
        demand_cost = demand_cost.apply(pd.to_numeric, errors='coerce')
        curtail_cost = curtail_cost.apply(pd.to_numeric, errors='coerce')
        
        # Create subplots
        fig, axes = plt.subplots(3, 2, figsize=(15, 15))
        fig.suptitle('Summary Results Visualization')
        
        # Plot system cost
        if not system_cost.empty and system_cost.notna().any().any():
            system_cost.plot(kind='bar', ax=axes[0,0], title='System Cost')
            axes[0,0].set_ylabel('Cost')
            axes[0,0].tick_params(axis='x', rotation=45)
        else:
            axes[0,0].set_title('System Cost - No numeric data available')
        
        # Plot price convergence
        if not price_conv.empty and price_conv.notna().any().any():
            price_conv.plot(kind='bar', ax=axes[0,1], title='Price Convergence')
            axes[0,1].set_ylabel('Price')
            axes[0,1].tick_params(axis='x', rotation=45)
        else:
            axes[0,1].set_title('Price Convergence - No numeric data available')
        
        # Plot premium up/down
        if not premium_up.empty and premium_up.notna().any().any():
            premium_up.plot(kind='bar', ax=axes[1,0], title='Premium Up')
            axes[1,0].set_ylabel('Premium')
            axes[1,0].tick_params(axis='x', rotation=45)
        else:
            axes[1,0].set_title('Premium Up - No numeric data available')
        
        if not premium_down.empty and premium_down.notna().any().any():
            premium_down.plot(kind='bar', ax=axes[1,1], title='Premium Down')
            axes[1,1].set_ylabel('Premium')
            axes[1,1].tick_params(axis='x', rotation=45)
        else:
            axes[1,1].set_title('Premium Down - No numeric data available')
        
        # Plot demand and curtailment costs
        if not demand_cost.empty and demand_cost.notna().any().any():
            demand_cost.plot(kind='bar', ax=axes[2,0], title='Demand Cost')
            axes[2,0].set_ylabel('Cost')
            axes[2,0].tick_params(axis='x', rotation=45)
        else:
            axes[2,0].set_title('Demand Cost - No numeric data available')
        
        if not curtail_cost.empty and curtail_cost.notna().any().any():
            curtail_cost.plot(kind='bar', ax=axes[2,1], title='Curtailment Cost')
            axes[2,1].set_ylabel('Cost')
            axes[2,1].tick_params(axis='x', rotation=45)
        else:
            axes[2,1].set_title('Curtailment Cost - No numeric data available')
        
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"Error creating visualizations: {str(e)}")

# Example usage for visualization
if __name__ == "__main__":
    summary_file = os.path.join('Test_output_files/Results_SectionV', f"Summary_Results_{date.today().strftime('%Y_%m_%d')}.xlsx")
    plot_summary_results(summary_file)


Error creating visualizations: [Errno 2] No such file or directory: 'Test_output_files/Results_SectionV\\Summary_Results_2025_02_27.xlsx'
